In [59]:
import os
import glob
import IPython.display as ipd
from IPython.display import display


import numpy as np
from scipy.stats import norm, kurtosis, skew

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

import librosa
import librosa.display as ld
from scipy import signal
import soundfile as sf

import pandas as pd
import torch

import gzip

In [57]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [7]:
# jupyter kernel interrupt sends SIGINT so need to implement this manually for some reason
import signal, sys
terminate = False

def signal_handling(signum, frame):
    global terminate
    terminate = True
    

In [ ]:
def load_signals(songs):
    with open('./signal_data/%s.csv'%song_id, 'r') as f:
        arr = np.loadtxt(f)
    arr.shape

In [32]:
# signal needs to be in this cell for the SIGINT to flow through (SIX YEAR OLD BUG??)
signal.signal(signal.SIGINT, signal_handling)

'''
running feature calculations on only 728 songs is already extremely cpu-intensive. probably have to break this up,
maybe about 500 songs at a time, write their signal/sr to csv, not sure how jupyter is handling garbage collection here

first we create our list of song ids
then the next loop we only run for every 500 songs at a time, then we set their metadata to the appropriate song
in the list, in order, so that when we then save that signal numpy array, in order to the hd5 format, we preserve
order and use that to associate between song ids/metadata and saved song music files
'''

songs = []
metadata = []

curr_count = 0
# first we just gather all the files that we have, set their id in the same place in the songs list
for fname in glob.glob("../youtubescraper/spotify_yt_data/*.flac"):
    if terminate:
        terminate=False
        break
    if curr_count == 5:
        break
    song_id = fname.split('/')[-1].split('.')[0]
    
    data, sr = sf.read(fname)
    
    # if stereo, we average the two channels of the signal to 1-dim
    if data.ndim == 2:
        data = (data[:,0] + data[:,1]) / 2
        channel = 2
    
    # now we build this song's metadata, then append it to our list which we will concatenate to metadata.csv afterwards
    meta = {
        'id': song_id,
        'sr': sr,
        'channel': channel
    }
    metadata.append(meta)
    
    with open('./signal_data/%s.csv'%song_id, 'w') as f:
        np.savetxt(f, data)

    curr_count += 1

In [66]:
# now we concatenate our signal metadata to the metadata file (we read old file then overwrite it)
new_meta_df = pd.DataFrame(metadata)
meta_df = pd.read_csv('./test_meta.csv')
meta_df = meta_df.merge(new_meta_df, on='id', how='left')
column_order = ['id', 'artist', 'title', 'duration', 'sr', 'channel']
meta_df.to_csv('./test_meta.csv', columns=column_order, encoding='utf-8', index=False)

In [67]:
# open metadata
df = pd.read_csv('./test_meta.csv')
df.iloc[:20]

,id,artist,title,duration,sr,channel
0,l_7TWVX4EOk,NaN,Anarchy in the U.K. Lullaby Versions of Sex Pi...,215,NaN,NaN
1,fRPNEtRakns,NaN,WITHOUT YOU (Workout Remix) by Power Music Wor...,221,NaN,NaN
2,BbgWSN95V8k,Rebirth Brass Band,Feel Like Funking It Up (Part II),410,NaN,NaN
3,hJsOd3PWcAM,Gregory Isaacs,Gregory Isaacs - Long Sentence,221,NaN,NaN
4,XQF69A3oGjM,Koffee,Koffee - W (Official Video) ft. Gunna,281,NaN,NaN
5,ucVMH9BVdws,Part Time,The Hunter's Daughter,212,NaN,NaN
6,gkQoCCsu_fA,Simian Mobile Disco,Simian Mobile Disco - Cerulean,410,NaN,NaN
7,KoKzN-krWUQ,Michael Mazochi,Jacob Row,298,NaN,NaN
8,51v5xPgIJ3Y,The Gravity Guild,"The Gravity Guild - ""Step Outside""",273,NaN,NaN
9,0ciD-nvljZw,NaN,Atlantic Five Jazz Band - Calm Christmas Piano...,3406,NaN,NaN


In [72]:
'''
TODO:
    only store the signals (as np arrays) in the hd5 files, named with their corresponding id
    metadata.csv will store ['id', 'artist', 'title', 'duration', 'sr', 'channel'], so we can just link
    the metadata with the song_id in the hd5 file
'''
df = pd.DataFrame(songs)
# max duration 8 minutes
df = df.loc[df['duration'] <= 480]

# min duration 3 minutes
df = df.loc[df['duration'] >= 180]

# remove rows where no signal was able to be extracted (at least one case of it due to being an interview)
df = df[df['signal'].map(lambda x: len(x) > 0)]
df.reset_index(inplace=True, drop=True)

# need to manually set dtype for hd5 saving
df['signal'] = df['signal'].apply(lambda col: pd.to_numeric(col, errors='coerce'))
df.dtypes

id          object
signal      object
sr           int64
channel      int64
artist      object
title       object
duration     int64
dtype: object

In [29]:
df.head()

,id,signal,sr,channel,artist,title,duration
0,scbeMwGaYJs,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,I-Wayne,I Wayne - I need her in my arms,216
1,dJ6I1WY-VAw,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,NaN,Check Yes or No - Lullaby Versions of George S...,332
2,EE94VcQ8xw0,"[-0.0002307295799255371, -0.000244915485382080...",48000,2,"Kevin Gates, Level",Get In There (feat. Level),243


In [9]:
'''

Here we get all the frequency level features. First they're computed using librosa,
then we reduce them to their mean, variance, kurtosis, skew.Of course these result in 
very general characteristics of each songs, but they provide a high level, quicker 
feature mapping for clustering; the CNN architecture will provide a more granular 
mapping of particular feature quirks and details.

we need to add a constant bias, primarily for zero crossing rate, to handle the silence
in the beginning of any song, which can have variable length before the music actually begins
for now we set this to 0.001, but we may have to revisit this to see if it affects anything else
which it shouldn't, as the distribution of freqs is the same.

'''
ZERO_CROSSING_RATE_BIAS = 0.001
def run_freq_stats(df):
    '''
    TODO: automate feature run in this function
    '''
    features = ['zcr', 'spectral_centroid', 'rolloff_frequency', 'mfcc', 'spectral_contrast', 'spectral_bandwidth', 'spectral_flatness']
    signal = df['signal']
    sr = df['sr']
    pass
test_df = df.copy()

# adding this bias is SUPER computationally costly in jupyter for some reason
# test_df['signal'] += ZERO_CROSSING_RATE_BIAS

In [10]:
df['title'].iloc[:10]

0                      I Wayne - I need her in my arms
1    Check Yes or No - Lullaby Versions of George S...
2                           Get In There (feat. Level)
3                            God Will Take Care of You
4                Fazlija - Gdje si mori - (Audio 2000)
5                           Jehst - High Plains Anthem
6    Anarchy in the U.K. Lullaby Versions of Sex Pi...
7    WITHOUT YOU (Workout Remix) by Power Music Wor...
8                    Feel Like Funking It Up (Part II)
9                       Gregory Isaacs - Long Sentence
Name: title, dtype: object

In [11]:
# getting zero-crossing rate
test_df['zcr'] = test_df['signal'].apply(lambda x: librosa.feature.zero_crossing_rate(x + ZERO_CROSSING_RATE_BIAS))

test_df['zcr_mean'] = test_df['zcr'].apply(lambda x: np.mean(x))
test_df['zcr_var'] = test_df['zcr'].apply(lambda x: np.var(x, ddof=1))
test_df['zcr_kurtosis'] = test_df['zcr'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['zcr_skew'] = test_df['zcr'].apply(lambda x: skew(np.ndarray.flatten(x)))

KeyboardInterrupt: 

In [ ]:
# getting spectral centroid
test_df['spectral_centroid'] = test_df.apply(lambda x: librosa.feature.spectral_centroid(y=x['signal'], sr=x['sr']), axis=1)

test_df['spectral_centroid_mean'] = test_df['spectral_centroid'].apply(lambda x: np.mean(x))
test_df['spectral_centroid_var'] = test_df['spectral_centroid'].apply(lambda x: np.var(x, ddof=1))
test_df['spectral_centroid_kurtosis'] = test_df['spectral_centroid'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['spectral_centroid_skew'] = test_df['spectral_centroid'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
# getting rolloff frequency, default percent is 0.85
test_df['rolloff_frequency'] = test_df.apply(lambda x: librosa.feature.spectral_rolloff(y=x['signal'], sr=x['sr']), axis=1)

test_df['rolloff_frequency_mean'] = test_df['rolloff_frequency'].apply(lambda x: np.mean(x))
test_df['rolloff_frequency_var'] = test_df['rolloff_frequency'].apply(lambda x: np.var(x, ddof=1))
test_df['rolloff_frequency_kurtosis'] = test_df['rolloff_frequency'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['rolloff_frequency_skew'] = test_df['rolloff_frequency'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
# getting mel coefficients
test_df['mfcc'] = test_df.apply(lambda x: librosa.feature.mfcc(y=x['signal'], sr=x['sr']), axis=1)

test_df['mfcc_mean'] = test_df['mfcc'].apply(lambda x: np.mean(x))
test_df['mfcc_var'] = test_df['mfcc'].apply(lambda x: np.var(x, ddof=1))
test_df['mfcc_kurtosis'] = test_df['mfcc'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['mfcc_skew'] = test_df['mfcc'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
# spectral contrast
test_df['spectral_contrast'] = test_df.apply(lambda x: librosa.feature.spectral_contrast(y=x['signal'], sr=x['sr']), axis=1)

test_df['spectral_contrast_mean'] = test_df['spectral_contrast'].apply(lambda x: np.mean(x))
test_df['spectral_contrast_var'] = test_df['spectral_contrast'].apply(lambda x: np.var(x, ddof=1))
test_df['spectral_contrast_kurtosis'] = test_df['spectral_contrast'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['spectral_contrast_skew'] = test_df['spectral_contrast'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
# spectral bandwidth
test_df['spectral_bandwidth'] = test_df.apply(lambda x: librosa.feature.spectral_bandwidth(y=x['signal'], sr=x['sr']), axis=1)

test_df['spectral_bandwidth_mean'] = test_df['spectral_bandwidth'].apply(lambda x: np.mean(x))
test_df['spectral_bandwidth_var'] = test_df['spectral_bandwidth'].apply(lambda x: np.var(x, ddof=1))
test_df['spectral_bandwidth_kurtosis'] = test_df['spectral_bandwidth'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['spectral_bandwidth_skew'] = test_df['spectral_bandwidth'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
# get spectral flatness
test_df['spectral_flatness'] = test_df['signal'].apply(librosa.feature.spectral_flatness)

test_df['spectral_flatness_mean'] = test_df['spectral_flatness'].apply(lambda x: np.mean(x))
test_df['spectral_flatness_var'] = test_df['spectral_flatness'].apply(lambda x: np.var(x, ddof=1))
test_df['spectral_flatness_kurtosis'] = test_df['spectral_flatness'].apply(lambda x: kurtosis(np.ndarray.flatten(x), fisher=True))
test_df['spectral_flatness_skew'] = test_df['spectral_flatness'].apply(lambda x: skew(np.ndarray.flatten(x)))

In [ ]:
test_df.columns

In [ ]:
test_df.drop(columns=[
    'zcr', 
    'spectral_centroid', 
    'rolloff_frequency', 
    'mfcc', 
    'spectral_contrast', 
    'spectral_bandwidth', 
    'spectral_flatness'], 
             inplace=True, axis=1)

In [ ]:
'''
standardizing everything and then running PCA, setting explained variance==.95 (we can range .95-.99)

note: need to compile literally all our data together before standardizing to perform proper PCA
'''
pca = PCA(n_components=3)
non_features = ['id', 'signal', 'sr', 'channel', 'artist', 'title', 'duration']

# df with features only
data = test_df[test_df.columns.difference(non_features)]

# standardize features
scaled_df = (data - data.mean()) / data.std()

In [ ]:
pc_components = pca.fit_transform(scaled_df)

principal_df = pd.DataFrame(data=pc_components, columns=['pca1', 'pca2', 'pca3'])
compiled_df = pd.concat([principal_df, test_df[['id']]], axis=1)

In [ ]:
compiled_df.head()

In [ ]:
model = KMeans(n_clusters=3)
model.fit(compiled_df[compiled_df.columns.difference(['id'])])
centroids = model.cluster_centers_

In [ ]:
ax1 = compiled_df.plot.scatter(x='pca1', y='pca2', c='pca3', colormap='viridis')
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
for i, label 

In [ ]:
def find_dist(centroid, y):
    dist = np.linalg.norm(centroid - y)
    return dist

In [ ]:
compiled_df['centroid_0_dist'] = compiled_df.apply(lambda x: find_dist(centroid=centroids[0], y=np.array([x['pca1'], x['pca2'], x['pca3']])), axis=1)
compiled_df['centroid_1_dist'] = compiled_df.apply(lambda x: find_dist(centroid=centroids[1], y=np.array([x['pca1'], x['pca2'], x['pca3']])), axis=1)
compiled_df['centroid_2_dist'] = compiled_df.apply(lambda x: find_dist(centroid=centroids[2], y=np.array([x['pca1'], x['pca2'], x['pca3']])), axis=1)

In [ ]:
print(compiled_df['centroid_0_dist'].min())
print(compiled_df['centroid_1_dist'].min())
print(compiled_df['centroid_2_dist'].min())
print()
print(compiled_df['centroid_0_dist'].max())
print(compiled_df['centroid_1_dist'].max())
print(compiled_df['centroid_2_dist'].max())

In [ ]:
first = compiled_df.loc[compiled_df['centroid_0_dist'] == compiled_df['centroid_0_dist'].min()]
second = compiled_df.loc[compiled_df['centroid_1_dist'] == compiled_df['centroid_1_dist'].min()]
third = compiled_df.loc[compiled_df['centroid_2_dist'] == compiled_df['centroid_2_dist'].min()]

In [ ]:
print(first, second, third)

In [ ]:
df.loc[df['id'] == 'ULmQ7E0mIKw']

In [ ]:
df.loc[df['id'] == 'DHcHGv0g200']

In [ ]:
df.loc[df['id'] == 'TMyQR9x5Qc4']

In [36]:
signal_df = df.copy()

In [37]:

'''
either we use CQT or mel spectrograms, not sure yet
'''
# compiling all the spectrograms here
def compile_spectrograms(df):
    specs = df.values.tolist()
    cols = df.columns
    print(specs[:10])
    print(cols)
    return
    for s in curr_songs:
    #     print(s['title'])
        signal, sr = s['signal'], s['sr']
        print(signal.shape)

        librosa.display.waveplot(signal, sr=sr)
        plt.show()

        X = librosa.stft(signal)
        X_db = librosa.amplitude_to_db(abs(X))
        plt.figure(figsize=(14, 5))
        ld.specshow(X_db, sr=sr, x_axis='time', y_axis='hz')
        plt.colorbar()

        plt.show()

compile_spectrograms(signal_df)



,id,signal,sr,channel,artist,title,duration
0,scbeMwGaYJs,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,I-Wayne,I Wayne - I need her in my arms,216
1,dJ6I1WY-VAw,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,NaN,Check Yes or No - Lullaby Versions of George S...,332
2,EE94VcQ8xw0,"[-0.0002307295799255371, -0.000244915485382080...",48000,2,"Kevin Gates, Level",Get In There (feat. Level),243
3,4EhZrkzGOUA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44100,2,The Echoaires,God Will Take Care of You,279
4,GcYRJhQ7bws,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,NaN,Fazlija - Gdje si mori - (Audio 2000),209
5,7vL0zhBdCk8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44100,2,Jehst,Jehst - High Plains Anthem,246
6,l_7TWVX4EOk,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,NaN,Anarchy in the U.K. Lullaby Versions of Sex Pi...,215
7,fRPNEtRakns,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,NaN,WITHOUT YOU (Workout Remix) by Power Music Wor...,221
8,BbgWSN95V8k,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44100,2,Rebirth Brass Band,Feel Like Funking It Up (Part II),410
9,hJsOd3PWcAM,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",48000,2,Gregory Isaacs,Gregory Isaacs - Long Sentence,221
